In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from math import sqrt

# -------------------
# CONFIG
# -------------------
# Put your universe here (e.g., NIFTY500 .NS tickers)
tickers = ["RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "INFY.NS", "ICICIBANK.NS"]  # <--- replace/add
benchmark = "^NSEI"   # NIFTY50 index
lookback_days = 252
use_log_returns = False         # set True if you prefer log returns for volatility
rank_by = "diff"                # "diff" => Stock VOLAR - NIFTY VOLAR, "ratio" => Stock VOLAR / NIFTY VOLAR
min_bars = 200                  # guardrail to drop thin history
save_csv = False                # set True to write CSV
csv_path = "volar_rank_52wh_filter.csv"

# -------------------
# FETCH DATA
# -------------------
end = dt.date.today()
start = end - dt.timedelta(days=lookback_days*3)  # extra buffer for missing days

symbols = list(dict.fromkeys(tickers + [benchmark]))  # de-dup while preserving order
px = yf.download(symbols, start=start, end=end, auto_adjust=True, progress=False)["Close"]
px = px.dropna(how="all")
# Ensure we have enough data
px = px.dropna(axis=0, how="any")  # strict: keep dates with all symbols for clean alignment

if px.shape[0] < lookback_days + 2:
    raise ValueError("Not enough aligned data for all symbols. Reduce strictness or add more history.")

# -------------------
# SLICE LAST 252 TRADING DAYS
# -------------------
window = px.iloc[-lookback_days-1:]   # include one extra row for daily returns calc
daily_ret = np.log(px/px.shift(1)) if use_log_returns else px.pct_change()
daily_ret = daily_ret.iloc[-lookback_days:]  # exactly 252 daily returns

# -------------------
# COMPUTE METRICS
# -------------------
# 252d Return
ret_252 = (px.iloc[-1] / px.iloc[-lookback_days] - 1).rename("Return_252d")

# Annualized Volatility
ann_vol = (daily_ret.std(axis=0) * sqrt(252)).rename("Ann_Vol")

# VOLAR = Return / Vol
volar = (ret_252 / ann_vol).rename("VOLAR")

# Benchmark values
nifty_ret = ret_252[benchmark]
nifty_vol = ann_vol[benchmark]
nifty_volar = volar[benchmark]

# Relative VOLAR vs NIFTY50
if rank_by == "ratio":
    rel_volar = (volar / nifty_volar).rename("Rel_VOLAR_vs_NIFTY")
else:
    rel_volar = (volar - nifty_volar).rename("Rel_VOLAR_vs_NIFTY")

# -------------------
# 52-WEEK HIGH FILTER (last 252 bars)
# -------------------
px_1y = px.iloc[-lookback_days:]
high_52w = px_1y.max().rename("High_52w")
last_price = px.iloc[-1].rename("Last")

within_20pct = (last_price / high_52w >= 0.80).rename("Within_20pct_52wHigh")

# -------------------
# BUILD TABLE
# -------------------
df = pd.concat([last_price, high_52w, ret_252, ann_vol, volar, rel_volar, within_20pct], axis=1)
df = df.drop(index=benchmark)  # drop benchmark row
df = df[df["Within_20pct_52wHigh"]]  # apply filter

# Sort by relative VOLAR (desc)
df = df.sort_values(by="Rel_VOLAR_vs_NIFTY", ascending=False)

# Optional: tidy display columns
df = df[[
    "Last", "High_52w",
    "Return_252d", "Ann_Vol", "VOLAR",
    "Rel_VOLAR_vs_NIFTY", "Within_20pct_52wHigh"
]]

# Pretty percentages for quick view (doesn't change underlying numbers)
display_df = df.copy()
for col in ["Return_252d", "Ann_Vol"]:
    display_df[col] = (display_df[col] * 100).round(2).astype(str) + "%"

print("\nRanked by Relative VOLAR vs NIFTY50 (descending):\n")
print(display_df.to_string())

if save_csv:
    df.to_csv(csv_path, index=True)
    print(f"\nSaved to {csv_path}")



Ranked by Relative VOLAR vs NIFTY50 (descending):

                     Last     High_52w Return_252d Ann_Vol     VOLAR  Rel_VOLAR_vs_NIFTY  Within_20pct_52wHigh
Ticker                                                                                                        
HDFCBANK.NS    950.299988  1012.900024       9.21%   17.8%  0.517200            0.923794                  True
ICICIBANK.NS  1348.099976  1477.201782       2.62%  17.39%  0.150558            0.557153                  True
RELIANCE.NS   1372.800049  1535.367920      -7.74%  21.07% -0.367493            0.039102                  True
